In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

print("TensorFlow version:", tf.__version__)
print("TensorFlow Probability version:", tfp.__version__)




TensorFlow version: 2.17.0
TensorFlow Probability version: 0.24.0


Context
1. Cold days are encoded by a 0 and hot days are encoded by a 1
2. The first day in our sequence has an 80% chance of being cold
3. A cold day has a 30% chance of being followed by a hot day
4. A hot day has a 20% chance of being followed by a cold day
5. On each day the temperature is normally distributed with mean and standard deviation 0 and 5 on a cold day and mean and standard deviation 15 and 10 on a hot day

We want to predict the average temperature

### Creating distribution variables

In [5]:
tfd = tfp.distributions #making a shortcut
initial_distribution = tfd.Categorical(probs=[0.8,0.2]) #refers to point 2, initial state
transition_distribution = tfd.Categorical(probs=[[0.7,0.3], #S1, probabilities of transitions
                                                  [0.2,0.8]]) #S2
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.]) #the loc represents the mean temperature and the scale is the standard desviation
#en el estado 1 la media de temperatura es 0, y en el estado 2 la media es 15
#en el estado 1 la desviacion estandar es 5, y en el estado 2 la desviacion estandar es 10


### Creating the hidden markov model

Un modelo oculto de Márkov o HMM es un modelo estadístico en el que se asume que el sistema a modelar es un proceso de Márkov de parámetros desconocidos. El objetivo es determinar los parámetros desconocidos de dicha cadena a partir de los parámetros observables.

In [9]:
model = tfd.HiddenMarkovModel(
    initial_distribution = initial_distribution, #probability distribution os the initial hidden states
    transition_distribution = transition_distribution, #probability distributions of the state transitions
    observation_distribution = observation_distribution, #how observations are distributed given the hidden state
    num_steps = 7 #how many days we want to predict
)

### Getting the expected temperatures

Symbolic Tensors: TensorFlow often deals with tensors symbolically, meaning they describe computations but do not have specific values until you run them within a session or an execution environment.
Deferred Computation: TensorFlow's computational graph allows for deferred computation. The actual values of tensors are only computed when you explicitly evaluate them in a session or within an eager execution environment.

In [11]:
mean = model.mean() #partially defined tensors
#devuelve un tensor que representa el valor esperado de la media de las observaciones segun el modelo

#partially defined tensor: tensor parcialmente definido (no tiene un valor concreto hasta que se evalua dentro de un contexto de ejecucion)

#due to the way TensorFlow works on a lower level we need to evaluate part of the graph
#from within a session to see the value of this tensor

with tf.compat.v1.Session() as sess: #crea una sesion de tensorflow dentro de un contexto de administracion de recursos
    print(mean.numpy()) #convierte el tensor en un array de numpy

[3.        5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ]


[3.        5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ] this are the expected temperatures for the following 7 days